# lab 1: Least Squares 

In [8]:
import pandas as pd 
import numpy as np

df  = pd.read_csv("train.csv")

print(df.head())

          x1         x2             y
0 -50.000000 -30.000000 -1.499390e+08
1 -49.849850 -29.819820 -1.508846e+08
2 -49.699700 -29.639640 -1.517676e+08
3 -49.549550 -29.459459 -1.525890e+08
4 -49.399399 -29.279279 -1.533500e+08


In [9]:
x1_all = df["x1"].to_numpy()
x2_all = df["x2"].to_numpy()
y_all  = df["y"].to_numpy()

split = 640

x1=x1_all[:split]
x2=x2_all[:split]
y =y_all[:split]

xt1=x1_all[split:]
xt2=x2_all[split:]
yt =y_all[split:]




- We define the function build_vandermonde(x1,x2,degree), where i combined all the vandermonde matrices until order 5 (just to make it cleaner).
- THen in a for loop that loops 5 times, (for each order), We train the data on these 5 using 80% of the data.
- we use the remaining 20% to test and see which of the coefficients fits best with the dat.
- Then we store the coefs of each order and the error in their corresponding lists. 
- We notice that the fitting with Vandermonds matrix till order  5, all have huge MSE values, up to 10e+19/20


In [10]:
def build_vandermonde(x1, x2, degree):

    if degree == 1:
        return np.column_stack([np.ones_like(x1), x1, x2])
    elif degree == 2:
        return np.column_stack([np.ones_like(x1),  x1, x2,x1**2, x1*x2, x2**2])
    elif degree == 3:
        return np.column_stack([np.ones_like(x1), x1, x2,x1**2, x1*x2, x2**2,x1**3, x1**2*x2, x1*x2**2, x2**3])
    elif degree == 4:
        return np.column_stack([np.ones_like(x1),  x1, x2,x1**2, x1*x2, x2**2,x1**3, x1**2*x2, x1*x2**2, x2**3,x1**4, x1**3*x2, x1**2*x2**2, x1*x2**3, x2**4])
    elif degree == 5:
         return np.column_stack([np.ones_like(x1), x1, x2, x1**2, x1*x2, x2**2,x1**3, x1**2*x2, x1*x2**2, x2**3, x1**4, x1**3*x2, x1**2*x2**2, x1*x2**3, x2**4, x1**5, x1**4*x2, x1**3*x2**2, x1**2*x2**3, x1*x2**4, x2**5])


In [11]:
mse_list = []
coef_list = []

for i in range(1, 6):
    V = build_vandermonde(x1, x2, i)
    Vt = build_vandermonde(xt1, xt2, i)
    
    #solving for coefs
    VT = V.T
    VTV = VT @ V
    VTy = VT @ y
    coefs = np.linalg.solve(VTV, VTy)
    
    #predicted y values
    yp = Vt @ coefs
    
    #mse y test and y predicted
    mse = np.mean((yt.reshape(-1, 1) - yp)**2)
    
    mse_list.append(mse)
    coef_list.append(coefs.ravel())

for i in range(5):
    print(f"Order {i+1}: MSE = {mse_list[i]}")

Order 1: MSE = 1.464898618537774e+20
Order 2: MSE = 1.0162301854393197e+20
Order 3: MSE = 6.657687093974446e+19
Order 4: MSE = 5.589138716828531e+19
Order 5: MSE = 6.793806797114696e+19


From the previous results we can clearly see that the best MSE has the fiiting of order 4, the fitting of the polynomial in order 5 starts overfiitng and we get a higher MSE.

however our Error is still huge  of order 10e+19, however it is still acceptable because of the huge magnitudes in the dataset.

we need eto scale the data to get more precise predictions, 


Now, before building the Vandermonde matrix, we scale the input variables $(x_1, x_2)$ to improve numerical stability.  
Polynomial terms of orders 4 or 5  can grow extremely large if the original data values are large, which makes the Vandermonde matrix basically blow up.


We transform each input feature to have a smaller, standardized range:
we use 

$$
x_i' = \frac{x_i - \mu_i}{\sigma_i} \quad \text{(z-score standardization)}
$$
Where $ \mu _i $ is the mean values of $x_i$ and $\sigma _i$ is the standrad deviation.

These scaled values $(x_1', x_2', y')$ are then used to build the Vandermonde  matrix for fitting and prediction.




- When predicting on new data, we apply the same scaling parameters computed from the training set.
- After computing $y'$ we need do rescale, by simply inversing the equation of y_scaled and finding y


In [12]:
#z score parameters
mu1 = np.mean(x1 )
sigma1 = np.std( x1)

mu2 =np.mean( x2)
sigma2 = np.std(x2)

y_mean = np.mean(y)
y_std = np.std(y )

#finding saled values
x1_scaled = (x1-mu1 ) / sigma1
x2_scaled =( x2- mu2 ) / sigma2
y_scaled= (y - y_mean) / y_std

xt1_scaled= (xt1 - mu1) / sigma1
xt2_scaled=(xt2- mu2)/ sigma2


order = 5 #change  order from 1->4 to see the huge mse that we get 
#Building vandermond matrices vor scaled values using order 5,
V = build_vandermonde(x1_scaled, x2_scaled, order) 
Vt = build_vandermonde(xt1_scaled, xt2_scaled, order)

# solving matrix equation for finding coefficents
VT = V.T
VTV = VT @ V
VTy = VT @ y_scaled.reshape(-1, 1)
coeffs = np.linalg.solve(VTV, VTy)

yp_scaled =  Vt @ coeffs

yp = yp_scaled* y_std + y_mean

mse = np.mean((yt.reshape(-1, 1) - yp)**2)

print(mse)


4.443799497851518


we get $mse \approx 4.44 $ wich is great, its 10e0. (for polynomial of order 5)

Import values from test.csv

we use the vandermonde matrix of order 5, and the coefficents found for that order, to apprixamte and predict the y_test values,(interstengly going from order 4 to order 5 , the mse got down from 10e+18 to 10e0)

 

using pandas we create a new csv file where, we add two additional columns (y scaled and y unscaled) on "test.csv" for predicted y values for the test data.

In [13]:
df_test  = pd.read_csv("test.csv")
x1_test = df_test["x1"].to_numpy()
x2_test= df_test["x2"].to_numpy()

In [16]:
V_test = build_vandermonde(x1_test,x2_test,5) 

y_test_unscaled = V_test @ coef_list[4] #values of predicted y with coefficenet found without using z-score scaling and using order 4 (the best one form 1-5)

y_test_scaled = V_test @ coeffs # values of y with coeffs foudn WITH  z score scaling (using order 5, or you can specify the order by changing the variable, however order =5 gives the best answer)

df_test['y unscled'] = y_test_unscaled

df_test['y scaled'] = y_test_scaled

df_test.to_csv("test_output.csv", index=False)


I think 10e+19 is still good as the first few digits of the number are the same, its just that the magnituddes of the y-values are huge. however having more precise vvalues and an error of $\approx 4$, is better than $10^{19}$.